# Importing Modules
Importing all modules necessary to run this program

In [13]:
import mysql.connector as mycon
from tabulate import tabulate
from qrcodereader import scan_qr_from_webcam
import qrcode

# Calling MySQL
Currently working on localhost -> Will be updated to AWS Host

In [14]:
print("Connecting to AWS Server...")
cob = mycon.connect(user="admin", 
                    password="<PASSWORD>", 
                    host="crud-app.c9ayiokeiq4o.ap-south-1.rds.amazonaws.com", 
                    database="crud"
)
print("Connection Successful!")

Connecting to AWS Server...
Connection Successful!


# Creating Read Function
This function will display all the books available in the library

In [22]:
def read(filter):
    tab = cob.cursor()
    match filter:
        case 1:
            tab.execute("SELECT * FROM books")
        case 2:
            tab.execute("SELECT * FROM books WHERE status = 'A'")
        case 3:
            tab.execute("SELECT * FROM books WHERE status = 'B'")
    
    bookLis = []
    for i in tab:
        bookLis.append(i)
    print(tabulate(bookLis, headers=["Book ID", "Name of Book", "Shelf", "Status"]))
read(1)

  Book ID  Name of Book           Shelf    Status
---------  ---------------------  -------  --------
     1001  The Jungle Book        1A       A
     1003  To Kill a Mockingbird  4A       B


# Creating Specific Read Function
This function will display specific books available in the library


**Values Taken From User:**
1. Book ID

In [18]:
def specificRead(bookID):
    tab = cob.cursor()
    tab.execute(f'SELECT * FROM books WHERE bookID = {bookID}')
    for i in tab:
        if i[3] == "A":
            print(f"Book Name: {i[1]}\nShelf: {i[2]}\nStatus: Available")
        elif i[3] == "B":
            print(f"Book Name: {i[1]}\nShelf: {i[2]}\nStatus: Unavailable")
specificRead(1001)

Book Name: The Jungle Book
Shelf: 1A
Status: Available


# Creating a Create Function
This function will insert books in the library using QR Code

In [21]:
def insertBook():
    qrval = scan_qr_from_webcam()
    try:
        qrval = qrval.split(",")
        bookId = qrval[0].strip()
        bookName = qrval[1].strip()
        shelf = qrval[2].strip()
        status = qrval[3].strip()
    
    except:
        print("Could Not Scan QR Code Properly, Try Insering Manually...")
        bookId = int(input("Enter Book ID: "))
        bookName = input("Enter Book Name: ")
        shelf = input("Enter Shelf: ")
        status = input("Enter Status: ")
    
    finally:
        tab = cob.cursor()
        tab.execute(f'INSERT INTO books VALUES ({bookId}, "{bookName}", "{shelf}", "{status}")')
        cob.commit()
        print("Successfully Added to Library")
insertBook()

Successfully Added to Library


# Create QR Code
This function will create QR codes for books which can be printed and stuck onto books for identification in the future.

In [ ]:
qrlist = []
def takeQRVals():
    n = int(input("Enter Number of QRs: "))
    for i in range (n):
        bookID = int(input("Enter Book ID: "))
        bookName = input("Enter Book Name: ")
        shelf = input("Enter Shelf: ")
        status = input("Enter Status: ")
        qrlist.append([bookID, bookName, shelf, status])

def createQR():
    takeQRVals()
    for i in range (len(qrlist)):
        data = f"{qrlist[i][0]},{qrlist[i][1]},{qrlist[i][2]},{qrlist[i][3]}"
        qr = qrcode.QRCode(
            version=1,
            error_correction=qrcode.constants.ERROR_CORRECT_L,
            box_size=10,
            border=4,
        )
        qr.add_data(data)  # Add data to QR code
        qr.make(fit=True)
        
        # Create an image of the QR Code
        qr_img = qr.make_image(fill="black", back_color="white")
        
        # Save the QR code as an image file
        qr_img.save(f"qrcode{i}.png")
        
        # Show the QR Code
        qr_img.show()     

createQR()

# Create Update Table Function
This function will update a cell from the table

In [ ]:
def update():
    bookID = int(input("Enter Book ID: "))
    choice = int(input("What Would You Like To Update?\n1. bookID\n2. bookName\n3. shelf\n4. status\nEnter your choice: "))
    match choice:
        case 1:
            column = "bookID"
            newval = int(input("Enter New Value: "))
            tab = cob.cursor()
            tab.execute(f'UPDATE books SET {column} = {newval} WHERE bookID = {bookID}')
        case 2:
            column = "bookName"
            newval = input("Enter New Value: ")
            tab = cob.cursor()
            tab.execute(f'UPDATE books SET {column} = "{newval}" WHERE bookID = {bookID}')
        case 3:
            column = "shelf"
            newval = input("Enter New Value: ")
            tab = cob.cursor()
            tab.execute(f'UPDATE books SET {column} = "{newval}" WHERE bookID = {bookID}')
        case 4:
            column = "status"
            newval = input("Enter New Value: ")
            tab = cob.cursor()
            tab.execute(f'UPDATE books SET {column} = "{newval}" WHERE bookID = {bookID}')
    cob.commit()
    print("Successfully Updated")
        
update()

# Create Delete Row Function
This function will delete a row from the table


In [ ]:
def delete(bookID):
    tab = cob.cursor()
    tab.execute('select bookid from books')
    bookids = []
    for i in tab:
        bookids.append(i[0])
    
    if bookID in bookids:
        tab.execute(f"DELETE FROM books WHERE bookID = {bookID}")
        cob.commit()
        print("Successfully Deleted")
    else:
        print("Invalid BookID")
delete(1002)
        